In [1119]:
import base64
import json
import hmac
import hashlib
import time

In [1120]:
type('abc')
type(b'abc')

str

bytes

In [1121]:
data = b'abc'
len(data)
data[0]  # each byte is a number 0-255
data[1]
data[2]

3

97

98

99

In [1122]:
type(data[0])  # bytes is like [int, int, ...]

chr(data[0])  # integer to ascii
chr(data[1])
chr(data[2])

int

'a'

'b'

'c'

In [1123]:
chr(97)
chr(98)
chr(99)

'a'

'b'

'c'

In [1124]:
chr(0)  # 0-31 are control chars
chr(1)
chr(2)
chr(15)
chr(16)
chr(255)  # last char of 8 bit ascii

'\x00'

'\x01'

'\x02'

'\x0f'

'\x10'

'ÿ'

\xHH is hexadecimal byte escape

HH = exactly 2 hex digits

Represents one byte (0–255)

In [1125]:
'\x41'  # hex 41 = dec 65
'\x42'
'\x43'

'A'

'B'

'C'

In [1126]:
type('\x41')
'\x41' == 'A'

str

True

In [1127]:
print('x41 is my best friend')
print('\x41 is my best friend')
print('\\x41 is my best friend')

x41 is my best friend
A is my best friend
\x41 is my best friend


In [1128]:
print('apple/orange')  # forward slash in ok

apple/orange


In [1129]:
print('apple\\orange')  # \\ -> \
print(r'apple\orange')  # raw string

apple\orange
apple\orange


In [1130]:
data  # bytes
data.decode()  # str

b'abc'

'abc'

In [1131]:
f'{97}'
f'{97:.2f}'  # format specification after `:`

f'{97551}'
f'{97551:,}'
f'{97551:,.2f}'

'97'

'97.00'

'97551'

'97,551'

'97,551.00'

In [1132]:
f'{0:b}'  # binary representation of integers
f'{1:b}'
f'{2:b}'
f'{3:b}'
f'{4:b}'
f'{18:b}'

'0'

'1'

'10'

'11'

'100'

'10010'

In [1133]:
f'{0:b}'  # exact binary
f'{0:5b}'  # min len = 5
f'{0:05b}'  # padding with 0
f'{18:05b}'  # no padding needed

'0'

'    0'

'00000'

'10010'

In [1134]:
for i in data:
  print(i, chr(i), f'{i:08b}')

97 a 01100001
98 b 01100010
99 c 01100011


In [1135]:
list(b'abc')  # bytes to decimal

list(b'vidu is my hero')

[97, 98, 99]

[118, 105, 100, 117, 32, 105, 115, 32, 109, 121, 32, 104, 101, 114, 111]

In [1136]:
bytes([97, 98, 99])  # decimal to bytes

bytes([
    118, 105, 100, 117, 32, 105, 115, 32, 109, 121, 32, 104, 101, 114, 111
])

b'abc'

b'vidu is my hero'

In [1137]:
' '.join(
    f'{i:08b}' for i in [
        118, 105, 100, 117, 32, 105, 115, 32, 109, 121, 32, 104, 101, 114, 111
    ]
)  # full binary sequence for `vidu is my hero`

'01110110 01101001 01100100 01110101 00100000 01101001 01110011 00100000 01101101 01111001 00100000 01101000 01100101 01110010 01101111'

In [1138]:
int('01110110', 2)  # binary to int
chr(int('01110110', 2))  # int to char

118

'v'

In [1139]:
binary_str = '01110110 01101001 01100100 01110101 00100000 01101001 01110011 00100000 01101101 01111001 00100000 01101000 01100101 01110010 01101111'

for byte_str in binary_str.split(' '):
  print(byte_str, int(byte_str, 2), chr(int(byte_str, 2)))

01110110 118 v
01101001 105 i
01100100 100 d
01110101 117 u
00100000 32  
01101001 105 i
01110011 115 s
00100000 32  
01101101 109 m
01111001 121 y
00100000 32  
01101000 104 h
01100101 101 e
01110010 114 r
01101111 111 o


# implementing base64 encoding/decoding from scratch

### goal of base64

encoding: bytes -> str

decoding: str -> bytes

only 64 symbols (`base64_symbols`) are allowed in encoded str

In [1140]:
import string
base64_symbols = string.ascii_uppercase + string.ascii_lowercase + string.digits + '+/'
base64_symbols
len(base64_symbols)  # 26 + 26 + 10 + 2

'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

64

### encoding: mapping nums to symbols

0–25  → A–Z  
26–51 → a–z  
52–61 → 0–9  
62    → +  
63    → /  

In [1141]:
for i, symbol in enumerate(base64_symbols):
  print(i, f'{i:06b}', symbol)  # 6 bits: 2^6 = 64 states (0-63 in decimal)

0 000000 A
1 000001 B
2 000010 C
3 000011 D
4 000100 E
5 000101 F
6 000110 G
7 000111 H
8 001000 I
9 001001 J
10 001010 K
11 001011 L
12 001100 M
13 001101 N
14 001110 O
15 001111 P
16 010000 Q
17 010001 R
18 010010 S
19 010011 T
20 010100 U
21 010101 V
22 010110 W
23 010111 X
24 011000 Y
25 011001 Z
26 011010 a
27 011011 b
28 011100 c
29 011101 d
30 011110 e
31 011111 f
32 100000 g
33 100001 h
34 100010 i
35 100011 j
36 100100 k
37 100101 l
38 100110 m
39 100111 n
40 101000 o
41 101001 p
42 101010 q
43 101011 r
44 101100 s
45 101101 t
46 101110 u
47 101111 v
48 110000 w
49 110001 x
50 110010 y
51 110011 z
52 110100 0
53 110101 1
54 110110 2
55 110111 3
56 111000 4
57 111001 5
58 111010 6
59 111011 7
60 111100 8
61 111101 9
62 111110 +
63 111111 /


In [1142]:
# data = b'vidu is my hero'  # 120 bits
# data = b'vidu is my boy'  # 112 bits
data = b'R'  # 8 bits
type(data)

bytes

In [1143]:
with open('i-o-m-bitmap16.png', 'rb') as f:
  data = f.read()

data
len(data)

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x04\x03\x00\x00\x00\xed\xdd\xe2R\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x0b\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x07tIME\x07\xd9\x06\x18\x0f\x06!\xf2g\x9dr\x00\x00\x00-PLTE\x00\x00\x00888HHH```xxx\x80\x80\x80\x98f3\xa8\xa8\xa8\xcb\x00\x00\xcb\x983\xcb\x98f\xff33\xff\xcbf\xff\xff\xcb\xff\xff\xff\x01\x8c\xa7\x90\x00\x00\x00\x01bKGD\x00\x88\x05\x1dH\x00\x00\x00rIDAT\x08[cx\x07\x05\x0c@\xfc\x00D\x800\xefI\x060\xe3\xc1\x9d\x9cL\x06\x10\x83W\xf0\xe4\x9c\x1c>\x86w\x0f.:\t\x1e;\x03d\xf0\n\x958\x89\x9c\xe2cxpGP\xa9Dg\x15+\x03\xaf.\x90ujA(\x03S\xd0\xde;2\xabX\xe3\x18X\xb57\xeb\x02\x05\xde1<\xd00\xb2\x01\n\x00\xcdaP\x06\t\x80\x0cd\x00\t\x80\xedb\x08\x85Z\n\x06\x00\xa5yK*dS2\xb3\x00\x00\x00\x00IEND\xaeB`\x82'

297

In [1144]:
from io import BytesIO
from PIL import Image
Image.open(BytesIO(data))

### step 1: bytes -> binary str

In [1145]:
binary_str = ''.join(f'{byte:08b}' for byte in data)
binary_str
len(binary_str)
len(binary_str) % 6

'100010010101000001001110010001110000110100001010000110100000101000000000000000000000000000001101010010010100100001000100010100100000000000000000000000000001000000000000000000000000000000010000000001000000001100000000000000000000000011101101110111011110001001010010000000000000000000000000000001000110011101000001010011010100000100000000000000001011000110001111000010111111110001100001000001010000000000000000000000000000100101110000010010000101100101110011000000000000000000001011000100110000000000000000000010110001001100000001000000001001101010011100000110000000000000000000000000000000011101110100010010010100110101000101000001111101100100000110000110000000111100000110001000011111001001100111100111010111001000000000000000000000000000101101010100000100110001010100010001010000000000000000000000000011100000111000001110000100100001001000010010000110000001100000011000000111100001111000011110001000000010000000100000001001100001100110001100111010100010101000101010001100101100000000000000001100101

2376

0

### ⚠️ if length of binary str is NOT multiple of 6, last chunk will have less than 6 bits

In [1146]:
# chunks not having 6 bits are PROBLEM
int('101', 2)  # encoding: chunk(3 bits) -> num
f'{5:06b}'  # decoding: num -> chunk(6 bits)

5

'000101'

In [1147]:
n = len(binary_str)
count_padded_bytes = ((-n % 24)//8)

print(f'{count_padded_bytes=}')

for i in range(count_padded_bytes):
  binary_str += f'{0:08b}'

count_padded_bytes=0


### step 2: split binary str in chunks of 6 bits

In [1148]:
# problem: split a str in parts of length x

fruits = 'applemangolemon'  # 3 words of 5 letters each

split_indices = list(range(0, len(fruits), 5))  # start, end, step
split_indices

for i in split_indices:
  print(i, i+5, fruits[i:i+5])

[0, 5, 10]

0 5 apple
5 10 mango
10 15 lemon


In [1149]:
chunks = [binary_str[i:i+6] for i in range(0, len(binary_str), 6)]
print(chunks)

['100010', '010101', '000001', '001110', '010001', '110000', '110100', '001010', '000110', '100000', '101000', '000000', '000000', '000000', '000000', '001101', '010010', '010100', '100001', '000100', '010100', '100000', '000000', '000000', '000000', '000001', '000000', '000000', '000000', '000000', '000000', '010000', '000001', '000000', '001100', '000000', '000000', '000000', '000011', '101101', '110111', '011110', '001001', '010010', '000000', '000000', '000000', '000000', '000001', '000110', '011101', '000001', '010011', '010100', '000100', '000000', '000000', '001011', '000110', '001111', '000010', '111111', '110001', '100001', '000001', '010000', '000000', '000000', '000000', '000000', '100101', '110000', '010010', '000101', '100101', '110011', '000000', '000000', '000000', '001011', '000100', '110000', '000000', '000000', '000010', '110001', '001100', '000001', '000000', '001001', '101010', '011100', '000110', '000000', '000000', '000000', '000000', '000000', '011101', '110100',

### step 3: chunks -> nums [0-63]

In [1150]:
nums = [int(chunk, 2) for chunk in chunks]
nums

[34,
 21,
 1,
 14,
 17,
 48,
 52,
 10,
 6,
 32,
 40,
 0,
 0,
 0,
 0,
 13,
 18,
 20,
 33,
 4,
 20,
 32,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 16,
 1,
 0,
 12,
 0,
 0,
 0,
 3,
 45,
 55,
 30,
 9,
 18,
 0,
 0,
 0,
 0,
 1,
 6,
 29,
 1,
 19,
 20,
 4,
 0,
 0,
 11,
 6,
 15,
 2,
 63,
 49,
 33,
 1,
 16,
 0,
 0,
 0,
 0,
 37,
 48,
 18,
 5,
 37,
 51,
 0,
 0,
 0,
 11,
 4,
 48,
 0,
 0,
 2,
 49,
 12,
 1,
 0,
 9,
 42,
 28,
 6,
 0,
 0,
 0,
 0,
 0,
 29,
 52,
 18,
 20,
 53,
 5,
 1,
 61,
 36,
 6,
 6,
 0,
 60,
 6,
 8,
 31,
 9,
 39,
 39,
 23,
 8,
 0,
 0,
 0,
 0,
 45,
 20,
 4,
 49,
 20,
 17,
 16,
 0,
 0,
 0,
 3,
 32,
 56,
 14,
 4,
 33,
 8,
 18,
 6,
 1,
 32,
 24,
 7,
 33,
 56,
 30,
 8,
 2,
 0,
 32,
 9,
 33,
 38,
 12,
 58,
 34,
 40,
 42,
 12,
 44,
 0,
 0,
 12,
 46,
 24,
 12,
 60,
 46,
 24,
 25,
 47,
 60,
 51,
 12,
 63,
 63,
 11,
 25,
 47,
 63,
 63,
 50,
 63,
 63,
 63,
 63,
 48,
 6,
 12,
 41,
 57,
 0,
 0,
 0,
 0,
 0,
 1,
 24,
 36,
 45,
 7,
 17,
 0,
 2,
 8,
 1,
 17,
 53,
 8,
 0,
 0,
 0,
 0,
 28,
 36,
 37,
 4,
 16,

### step 4: nums -> symbols

In [1151]:
encoded_str = ''.join([base64_symbols[num] for num in nums])
encoded_str

'iVBORw0KGgoAAAANSUhEUgAAABAAAAAQBAMAAADt3eJSAAAABGdBTUEAALGPC/xhBQAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAd0SU1FB9kGGA8GIfJnnXIAAAAtUExURQAAADg4OEhISGBgYHh4eICAgJhmM6ioqMsAAMuYM8uYZv8zM//LZv//y////wGMp5AAAAABYktHRACIBR1IAAAAcklEQVQIW2N4BwUMQPwARIAw70kGMOPBnZxMBhCDV/DknBw+hncPLjoJHjsDZPAKlTiJnOJjeHBHUKlEZxUrA68ukHVqQSgDU9DeOzKrWOMYWLU36wIF3jE80DCyAQoAzWFQBgmADGQACYDtYgiFWgoGAKV5SypkUzKzAAAAAElFTkSuQmCC'

In [1152]:
# count_padded_bytes: 'A' at end -> '='
if count_padded_bytes > 0:
  encoded_str = encoded_str[:-count_padded_bytes] + '='*count_padded_bytes
encoded_str

'iVBORw0KGgoAAAANSUhEUgAAABAAAAAQBAMAAADt3eJSAAAABGdBTUEAALGPC/xhBQAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAd0SU1FB9kGGA8GIfJnnXIAAAAtUExURQAAADg4OEhISGBgYHh4eICAgJhmM6ioqMsAAMuYM8uYZv8zM//LZv//y////wGMp5AAAAABYktHRACIBR1IAAAAcklEQVQIW2N4BwUMQPwARIAw70kGMOPBnZxMBhCDV/DknBw+hncPLjoJHjsDZPAKlTiJnOJjeHBHUKlEZxUrA68ukHVqQSgDU9DeOzKrWOMYWLU36wIF3jE80DCyAQoAzWFQBgmADGQACYDtYgiFWgoGAKV5SypkUzKzAAAAAElFTkSuQmCC'

In [1153]:
base64.b64encode(data)  # python implementation

b'iVBORw0KGgoAAAANSUhEUgAAABAAAAAQBAMAAADt3eJSAAAABGdBTUEAALGPC/xhBQAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAd0SU1FB9kGGA8GIfJnnXIAAAAtUExURQAAADg4OEhISGBgYHh4eICAgJhmM6ioqMsAAMuYM8uYZv8zM//LZv//y////wGMp5AAAAABYktHRACIBR1IAAAAcklEQVQIW2N4BwUMQPwARIAw70kGMOPBnZxMBhCDV/DknBw+hncPLjoJHjsDZPAKlTiJnOJjeHBHUKlEZxUrA68ukHVqQSgDU9DeOzKrWOMYWLU36wIF3jE80DCyAQoAzWFQBgmADGQACYDtYgiFWgoGAKV5SypkUzKzAAAAAElFTkSuQmCC'

# decoding
### step 1: symbols -> nums

In [1154]:
# count_padded_bytes: '=' at end -> 'A'
encoded_str_og = encoded_str
encoded_str = encoded_str.replace("=", "A")

In [1155]:
nums = [base64_symbols.index(symbol) for symbol in encoded_str]
nums

[34,
 21,
 1,
 14,
 17,
 48,
 52,
 10,
 6,
 32,
 40,
 0,
 0,
 0,
 0,
 13,
 18,
 20,
 33,
 4,
 20,
 32,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 16,
 1,
 0,
 12,
 0,
 0,
 0,
 3,
 45,
 55,
 30,
 9,
 18,
 0,
 0,
 0,
 0,
 1,
 6,
 29,
 1,
 19,
 20,
 4,
 0,
 0,
 11,
 6,
 15,
 2,
 63,
 49,
 33,
 1,
 16,
 0,
 0,
 0,
 0,
 37,
 48,
 18,
 5,
 37,
 51,
 0,
 0,
 0,
 11,
 4,
 48,
 0,
 0,
 2,
 49,
 12,
 1,
 0,
 9,
 42,
 28,
 6,
 0,
 0,
 0,
 0,
 0,
 29,
 52,
 18,
 20,
 53,
 5,
 1,
 61,
 36,
 6,
 6,
 0,
 60,
 6,
 8,
 31,
 9,
 39,
 39,
 23,
 8,
 0,
 0,
 0,
 0,
 45,
 20,
 4,
 49,
 20,
 17,
 16,
 0,
 0,
 0,
 3,
 32,
 56,
 14,
 4,
 33,
 8,
 18,
 6,
 1,
 32,
 24,
 7,
 33,
 56,
 30,
 8,
 2,
 0,
 32,
 9,
 33,
 38,
 12,
 58,
 34,
 40,
 42,
 12,
 44,
 0,
 0,
 12,
 46,
 24,
 12,
 60,
 46,
 24,
 25,
 47,
 60,
 51,
 12,
 63,
 63,
 11,
 25,
 47,
 63,
 63,
 50,
 63,
 63,
 63,
 63,
 48,
 6,
 12,
 41,
 57,
 0,
 0,
 0,
 0,
 0,
 1,
 24,
 36,
 45,
 7,
 17,
 0,
 2,
 8,
 1,
 17,
 53,
 8,
 0,
 0,
 0,
 0,
 28,
 36,
 37,
 4,
 16,

### step 2: nums -> binary chunks of 6 bits

In [1156]:
chunks = [f'{num:06b}' for num in nums]
print(chunks)

['100010', '010101', '000001', '001110', '010001', '110000', '110100', '001010', '000110', '100000', '101000', '000000', '000000', '000000', '000000', '001101', '010010', '010100', '100001', '000100', '010100', '100000', '000000', '000000', '000000', '000001', '000000', '000000', '000000', '000000', '000000', '010000', '000001', '000000', '001100', '000000', '000000', '000000', '000011', '101101', '110111', '011110', '001001', '010010', '000000', '000000', '000000', '000000', '000001', '000110', '011101', '000001', '010011', '010100', '000100', '000000', '000000', '001011', '000110', '001111', '000010', '111111', '110001', '100001', '000001', '010000', '000000', '000000', '000000', '000000', '100101', '110000', '010010', '000101', '100101', '110011', '000000', '000000', '000000', '001011', '000100', '110000', '000000', '000000', '000010', '110001', '001100', '000001', '000000', '001001', '101010', '011100', '000110', '000000', '000000', '000000', '000000', '000000', '011101', '110100',

### step 3: join those chunks to get full binary str

In [1157]:
binary_str = ''.join(chunks)
binary_str

'100010010101000001001110010001110000110100001010000110100000101000000000000000000000000000001101010010010100100001000100010100100000000000000000000000000001000000000000000000000000000000010000000001000000001100000000000000000000000011101101110111011110001001010010000000000000000000000000000001000110011101000001010011010100000100000000000000001011000110001111000010111111110001100001000001010000000000000000000000000000100101110000010010000101100101110011000000000000000000001011000100110000000000000000000010110001001100000001000000001001101010011100000110000000000000000000000000000000011101110100010010010100110101000101000001111101100100000110000110000000111100000110001000011111001001100111100111010111001000000000000000000000000000101101010100000100110001010100010001010000000000000000000000000011100000111000001110000100100001001000010010000110000001100000011000000111100001111000011110001000000010000000100000001001100001100110001100111010100010101000101010001100101100000000000000001100101

### step 4: binary str -> bytes

each byte = 8 bits  

full process:
  1. split in chunks of 8 bits
  2. 8 bit chunk -> decimal num [0-255]
  3. step 2 is effectively: chunks -> [int, int, ...]
  4. finally: bytes([int, int, ...])

enjoy :)

In [1158]:
data = bytes(int(binary_str[i:i+8], 2) for i in range(0, len(binary_str), 8))
data

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x04\x03\x00\x00\x00\xed\xdd\xe2R\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x0b\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x07tIME\x07\xd9\x06\x18\x0f\x06!\xf2g\x9dr\x00\x00\x00-PLTE\x00\x00\x00888HHH```xxx\x80\x80\x80\x98f3\xa8\xa8\xa8\xcb\x00\x00\xcb\x983\xcb\x98f\xff33\xff\xcbf\xff\xff\xcb\xff\xff\xff\x01\x8c\xa7\x90\x00\x00\x00\x01bKGD\x00\x88\x05\x1dH\x00\x00\x00rIDAT\x08[cx\x07\x05\x0c@\xfc\x00D\x800\xefI\x060\xe3\xc1\x9d\x9cL\x06\x10\x83W\xf0\xe4\x9c\x1c>\x86w\x0f.:\t\x1e;\x03d\xf0\n\x958\x89\x9c\xe2cxpGP\xa9Dg\x15+\x03\xaf.\x90ujA(\x03S\xd0\xde;2\xabX\xe3\x18X\xb57\xeb\x02\x05\xde1<\xd00\xb2\x01\n\x00\xcdaP\x06\t\x80\x0cd\x00\t\x80\xedb\x08\x85Z\n\x06\x00\xa5yK*dS2\xb3\x00\x00\x00\x00IEND\xaeB`\x82'

In [1159]:
if count_padded_bytes > 0:
  data = data[:-count_padded_bytes]
data

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x04\x03\x00\x00\x00\xed\xdd\xe2R\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x0b\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x07tIME\x07\xd9\x06\x18\x0f\x06!\xf2g\x9dr\x00\x00\x00-PLTE\x00\x00\x00888HHH```xxx\x80\x80\x80\x98f3\xa8\xa8\xa8\xcb\x00\x00\xcb\x983\xcb\x98f\xff33\xff\xcbf\xff\xff\xcb\xff\xff\xff\x01\x8c\xa7\x90\x00\x00\x00\x01bKGD\x00\x88\x05\x1dH\x00\x00\x00rIDAT\x08[cx\x07\x05\x0c@\xfc\x00D\x800\xefI\x060\xe3\xc1\x9d\x9cL\x06\x10\x83W\xf0\xe4\x9c\x1c>\x86w\x0f.:\t\x1e;\x03d\xf0\n\x958\x89\x9c\xe2cxpGP\xa9Dg\x15+\x03\xaf.\x90ujA(\x03S\xd0\xde;2\xabX\xe3\x18X\xb57\xeb\x02\x05\xde1<\xd00\xb2\x01\n\x00\xcdaP\x06\t\x80\x0cd\x00\t\x80\xedb\x08\x85Z\n\x06\x00\xa5yK*dS2\xb3\x00\x00\x00\x00IEND\xaeB`\x82'

In [1160]:
Image.open(BytesIO(data))

In [1161]:
base64.b64decode(encoded_str_og)  # python implementation

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x04\x03\x00\x00\x00\xed\xdd\xe2R\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x0b\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x07tIME\x07\xd9\x06\x18\x0f\x06!\xf2g\x9dr\x00\x00\x00-PLTE\x00\x00\x00888HHH```xxx\x80\x80\x80\x98f3\xa8\xa8\xa8\xcb\x00\x00\xcb\x983\xcb\x98f\xff33\xff\xcbf\xff\xff\xcb\xff\xff\xff\x01\x8c\xa7\x90\x00\x00\x00\x01bKGD\x00\x88\x05\x1dH\x00\x00\x00rIDAT\x08[cx\x07\x05\x0c@\xfc\x00D\x800\xefI\x060\xe3\xc1\x9d\x9cL\x06\x10\x83W\xf0\xe4\x9c\x1c>\x86w\x0f.:\t\x1e;\x03d\xf0\n\x958\x89\x9c\xe2cxpGP\xa9Dg\x15+\x03\xaf.\x90ujA(\x03S\xd0\xde;2\xabX\xe3\x18X\xb57\xeb\x02\x05\xde1<\xd00\xb2\x01\n\x00\xcdaP\x06\t\x80\x0cd\x00\t\x80\xedb\x08\x85Z\n\x06\x00\xa5yK*dS2\xb3\x00\x00\x00\x00IEND\xaeB`\x82'

In [1162]:
# base64.urlsafe_b64encode(b'a')  # 8 bits data (6, 2) -> 2
# base64.urlsafe_b64encode(b'ab')  # 16 bits data (6, 6, 4) -> 3
# base64.urlsafe_b64encode(b'abc')  # 24 bits data (6, 6, 6, 6) -> 4
# base64.urlsafe_b64encode(b'abcd')  # 32 bits data (6, 6, 6, 6, 6, 2) -> 6